In [ ]:
!pip install langchain_community
!pip install openai
!pip install IPython
!pip install pypdf
!pip install chromadb
!pip install -U langchain-openai

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
import json
import requests
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
file_name = '/content/drive/MyDrive/RAG Programs/RAG Coding/RAG Single File/config.json'

with open(file_name, 'r') as file:
  config = json.load(file)
  os.environ['OPENAI_API_KEY'] = config.get('API_KEY')
  os.environ["OPENAI_BASE_URL"] = config.get("OPENAI_API_BASE")

In [ ]:
#@title Load the PDF file

DOC_PATH = "/content/drive/MyDrive/RAG Programs/RAG Coding/RAG Single File/presentation_on_ai.pdf"
CHROMA_PATH = "ai-overview-db"

loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = text_splitter.split_documents(pages)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(api_key=os.environ.get("OPENAI_API_KEY"))

db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

In [ ]:
query = 'what is prompt engineering'

docs_chroma = db_chroma.similarity_search_with_score(query, k=10)

context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

In [ ]:
PROMPT_TEMPLATE = """
{context}
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question = query)
print(prompt)